# **Transfer Learning Algorithm**

In [ ]:
####################################### importing important libraries#######################################


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical
import librosa
import keras
from keras.layers import LSTM, Dense, Dropout, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras
from keras.layers import LSTM, Dense, Dropout, Flatten
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
import tensorflow as tf

Loading the model whose layers are to be used

In [ ]:
model2=keras.models.load_model('/content/drive/MyDrive/ML_A/Tri train/3. Tri-S/CRNN_80.h5')

In [ ]:
model2.summary()

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 16, 8, 64)         640       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 8, 4, 64)          0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 8, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 4, 2, 128)         0         
_________________________________________________________________
reshape_13 (Reshape)         (None, 8, 128)            0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 32)                20608     
_________________________________________________________________
flatten_25 (Flatten)         (None, 32)              

In [ ]:
len(model2.layers)
for i in range(7):
    model2.layers[i].trainable = False

Adding 3 more dense layers

In [ ]:
import tensorflow as tf

ll = model2.layers[7].output
ll = Dense(128)(ll)
ll = Dense(64)(ll)
ll = Dense(32)(ll)
ll = Dense(1,activation="sigmoid")(ll)

new_model = tf.keras.Model(inputs=model2.input,outputs=ll)

In [ ]:
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26_input (InputLayer) [(None, 16, 8, 1)]        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 16, 8, 64)         640       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 8, 4, 64)          0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 8, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 4, 2, 128)         0         
_________________________________________________________________
reshape_13 (Reshape)         (None, 8, 128)            0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 32)                20608 

Feature extraction

In [ ]:
label = []

import glob
def parser(row):
    # Function to load files and extract features
    f = glob.glob(row+'/'+'*.wav')
    print(len(f))
    feature = []
    for i in f:
        #file_name = '/content/drive/MyDrive/Dataset/audio/fold' + str(df["fold"][i]) + '/' + df["slice_file_name"][i]
        mels, sample_rate = librosa.load(i, res_type='kaiser_fast') 
        #print(mels.shape)
        x = []
        
        mels = np.mean(librosa.feature.melspectrogram(y=mels, sr=sample_rate).T,axis=0) 
        for i in range(0, len(mels), 128):
            if len(mels)-i<128:
              x.append(np.array(list(mels[i:])+[0]*(128-len(mels)+i)))
            else:
              x.append(mels[i:i+128])
 
        #print(mels.shape)  
        
        feature=np.array(list(x)+list(feature))
        label.append(row[-4:-1])

    return [feature, label]

Training Dataset feature extraction

In [ ]:
f, l= parser('/content/drive/MyDrive/cats_dogs/train/cat')
X = list(f)
y = [0]*len(X)

125


In [ ]:
f, l= parser('/content/drive/MyDrive/cats_dogs/train/dog')
X = X+list(f)
y = y+[1]*len(f)

85


In [ ]:
X=np.array(X)
y=np.array(y)

In [ ]:
X=X.reshape(210,16,8,1)

In [ ]:
X__train, X__test, y__train, y__test = train_test_split(X, y, test_size=0.3)

Cat: 0
Dog: 1

In [ ]:
y__train

array([0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0])

With 25 epochs


In [ ]:
callbacks = [ModelCheckpoint('/content/drive/MyDrive/ML_A/CNN_transfer learning2.h5', monitor='val_loss', mode='min', save_best_only=True)]
opt = Adam(lr=0.1)
new_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
history = new_model.fit(X__train, y__train, epochs = 25, batch_size = 64, validation_data = (X__test, y__test), callbacks=callbacks)


Epoch 1/25
3/3 [==============================] - 3s 420ms/step - loss: 16.6767 - accuracy: 0.4067 - val_loss: 905.9744 - val_accuracy: 0.4921
Epoch 2/25
3/3 [==============================] - 0s 33ms/step - loss: 843.0305 - accuracy: 0.4534 - val_loss: 96.2193 - val_accuracy: 0.5397
Epoch 3/25
3/3 [==============================] - 0s 37ms/step - loss: 278.3945 - accuracy: 0.5348 - val_loss: 9.4626 - val_accuracy: 0.6667
Epoch 4/25
3/3 [==============================] - 0s 34ms/step - loss: 47.2823 - accuracy: 0.4496 - val_loss: 261.9543 - val_accuracy: 0.5238
Epoch 5/25
3/3 [==============================] - 0s 34ms/step - loss: 152.0935 - accuracy: 0.5251 - val_loss: 42.9976 - val_accuracy: 0.5556
Epoch 6/25
3/3 [==============================] - 0s 43ms/step - loss: 42.4857 - accuracy: 0.6005 - val_loss: 12.8935 - val_accuracy: 0.6190
Epoch 7/25
3/3 [==============================] - 0s 33ms/step - loss: 27.5637 - accuracy: 0.4238 - val_loss: 12.0017 - val_accuracy: 0.6984
Epoch 8/

Testing on Real Recorded Dataset from youtube

In [ ]:
f, l= parser('/content/drive/MyDrive/ML_N/Dog_bark')
X_test = list(f)
y_test = [1]*len(X_test)
X_test=np.array(X_test)
X_test =  X_test.reshape(len(X_test), 16, 8, 1)
y_pred = new_model.predict(X_test)
y_pred
c = 0
for i in y_pred:
  if i<0.5:
    print(0)
  else:
    c+=1
    print(1)
print(c/len(y_pred))

76
1
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
0
0
0
0
1
0
0
1
1
0
1
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
1
1
1
1
1
0.7763157894736842


Testing Accuracy is 77.6%


.

Now Training on a normal CNN Model 

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape = (16, 8, 1)))
model.add(MaxPool2D(pool_size=(2, 2)))
#model.add(Dropout(0.1))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "tanh"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(1024, activation = "tanh"))
model.add(Dense(1, activation = "sigmoid"))

In [ ]:
opt = Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
#callbacks = [ModelCheckpoint('/content/drive/MyDrive/CNN_base.h5', monitor='val_loss', mode='min', save_best_only=True),EarlyStopping(monitor = 'val_loss',min_delta = 0,patience = 40,restore_best_weights = True)]
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 8, 64)         640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 4, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 2, 128)         0         
_________________________________________________________________
dropout (Dropout)            (None, 4, 2, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              1

In [ ]:
h = model.fit(X__train, y__train, epochs = 15, batch_size = 64, validation_data = (X__test, y__test))

Epoch 1/15
3/3 [==============================] - 1s 155ms/step - loss: 1.1348 - accuracy: 0.4992 - val_loss: 0.8093 - val_accuracy: 0.6825
Epoch 2/15
3/3 [==============================] - 0s 60ms/step - loss: 0.7102 - accuracy: 0.6576 - val_loss: 0.3428 - val_accuracy: 0.8730
Epoch 3/15
3/3 [==============================] - 0s 65ms/step - loss: 0.3768 - accuracy: 0.9026 - val_loss: 0.3961 - val_accuracy: 0.8254
Epoch 4/15
3/3 [==============================] - 0s 60ms/step - loss: 0.2758 - accuracy: 0.9318 - val_loss: 0.4379 - val_accuracy: 0.8254
Epoch 5/15
3/3 [==============================] - 0s 62ms/step - loss: 0.2939 - accuracy: 0.8855 - val_loss: 0.2857 - val_accuracy: 0.8889
Epoch 6/15
3/3 [==============================] - 0s 70ms/step - loss: 0.2114 - accuracy: 0.9255 - val_loss: 0.3235 - val_accuracy: 0.8571
Epoch 7/15
3/3 [==============================] - 0s 64ms/step - loss: 0.1491 - accuracy: 0.9557 - val_loss: 0.2796 - val_accuracy: 0.8730
Epoch 8/15
3/3 [==========

Testing on Same youtube Dataset

In [ ]:
#f, l= parser('/content/drive/MyDrive/ML_N/Dog_bark')
X_test = list(f)
y_test = [1]*len(X_test)
X_test=np.array(X_test)
X_test =  X_test.reshape(len(X_test), 16, 8, 1)
y_pred = model.predict(X_test)
y_pred
c = 0
for i in y_pred:
  if i<0.5:
    print(0)
  else:
    c+=1
    print(1)
print(c/len(y_pred))

1
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
0
0
0
0
0
1
1
1
1
1
0.32894736842105265


Testing Accuracy is 32.8%